In [1]:
import pandas as pd
from pathlib import Path

In [2]:
# now we implement a model that will predict based on the metadata given in the original data. 
#source: https://www.kaggle.com/redwankarimsony/power-of-metadata-xgboost-cnn-ensemble


!pip install xgboost
import xgboost as xgb
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
import keras


2025-08-23 11:14:48.359084: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1755947688.373979       8 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1755947688.378707       8 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-08-23 11:14:48.397453: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
train = pd.read_csv('../input/siim-isic-melanoma-classification/train.csv')
test = pd.read_csv('../input/siim-isic-melanoma-classification/test.csv')
sub = pd.read_csv('../input/siim-isic-melanoma-classification/sample_submission.csv')
train.head()


,image_name,patient_id,sex,age_approx,anatom_site_general_challenge,diagnosis,benign_malignant,target
0,ISIC_2637011,IP_7279968,male,45.0,head/neck,unknown,benign,0
1,ISIC_0015719,IP_3075186,female,45.0,upper extremity,unknown,benign,0
2,ISIC_0068279,IP_6890425,female,45.0,head/neck,unknown,benign,0
3,ISIC_0074268,IP_8723313,female,55.0,upper extremity,unknown,benign,0
4,ISIC_0074311,IP_2950485,female,40.0,lower extremity,unknown,benign,0


In [4]:
train['sex'] = train['sex'].astype("category").cat.codes +1
train['anatom_site_general_challenge'] = train['anatom_site_general_challenge'].astype("category").cat.codes +1
test['sex'] = test['sex'].astype("category").cat.codes +1
test['anatom_site_general_challenge'] = test['anatom_site_general_challenge'].astype("category").cat.codes +1
train.head()


,image_name,patient_id,sex,age_approx,anatom_site_general_challenge,diagnosis,benign_malignant,target
0,ISIC_2637011,IP_7279968,2,45.0,1,unknown,benign,0
1,ISIC_0015719,IP_3075186,1,45.0,6,unknown,benign,0
2,ISIC_0068279,IP_6890425,1,45.0,1,unknown,benign,0
3,ISIC_0074268,IP_8723313,1,55.0,6,unknown,benign,0
4,ISIC_0074311,IP_2950485,1,40.0,2,unknown,benign,0


In [5]:
train['sex'] = train['sex'].fillna('male')
train['age_approx'] = train['age_approx'].fillna(train['age_approx'].mean())
train['anatom_site_general_challenge'] = train['anatom_site_general_challenge'].fillna('torso')
test['sex'] = test['sex'].fillna('male')
test['age_approx'] = test['age_approx'].fillna(train['age_approx'].mean())
test['anatom_site_general_challenge'] = test['anatom_site_general_challenge'].fillna('torso')
train.head()


,image_name,patient_id,sex,age_approx,anatom_site_general_challenge,diagnosis,benign_malignant,target
0,ISIC_2637011,IP_7279968,2,45.0,1,unknown,benign,0
1,ISIC_0015719,IP_3075186,1,45.0,6,unknown,benign,0
2,ISIC_0068279,IP_6890425,1,45.0,1,unknown,benign,0
3,ISIC_0074268,IP_8723313,1,55.0,6,unknown,benign,0
4,ISIC_0074311,IP_2950485,1,40.0,2,unknown,benign,0


In [6]:
#to have a more balanced dataset, we create a new dataframe that contains a more equal percentage of each type of target image
df_0=train[train['target']==0].sample(600)
df_1=train[train['target']==1]
train=pd.concat([df_0,df_1])
train=train.reset_index()


In [7]:
from sklearn.model_selection import train_test_split

x_train_df = train[['sex', 'age_approx','anatom_site_general_challenge']]
y_train_df = train['target']


x_train, x_val, y_train, y_val = train_test_split(x_train_df, y_train_df, test_size=0.2, random_state=42)


x_test = test[['sex', 'age_approx','anatom_site_general_challenge']]


train_DMatrix = xgb.DMatrix(x_train, label= y_train)
test_DMatrix = xgb.DMatrix(x_test)


In [8]:
clf = xgb.XGBClassifier(n_estimators=2000, 
                        max_depth=8, 
                        objective='multi:softprob',
                        seed=0,  
                        nthread=-1, 
                        learning_rate=0.15, 
                        num_class = 2, 
                        scale_pos_weight = (32542/584))


In [9]:
clf.fit(x_train, y_train)


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:160: UserWarning: [11:14:53] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "scale_pos_weight" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.15, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=8, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=2000, n_jobs=None, nthread=-1,
              num_class=2, ...)

In [10]:
preds = clf.predict_proba(x_val)[:,1]
target = [round(value) for value in preds]


In [11]:
# evaluate predictions
accuracy = accuracy_score(y_val, target)
print("Accuracy: %.2f%%" % (accuracy * 100.0))


Accuracy: 61.43%


In [12]:
#the submission file:
test_df = test[['sex', 'age_approx','anatom_site_general_challenge']]
preds = clf.predict_proba(test_df)[:,1]
target = [round(value) for value in preds]


In [13]:
sub['target']=target
sub.to_csv('submission.csv', index=False)
sub.head()


,image_name,target
0,ISIC_0052212,0
1,ISIC_0076545,0
2,ISIC_0085172,0
3,ISIC_0086709,0
4,ISIC_0109568,0
